In [1]:
import cv2
import numpy as np
import os

data_dir = "dataset"
image_size = (64, 64)
X, y = [], []

for label, sign in enumerate(os.listdir(data_dir)):
    for img_name in os.listdir(os.path.join(data_dir, sign)):
        img_path = os.path.join(data_dir, sign, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, image_size)
        X.append(img)
        y.append(label)

X = np.array(X) / 255.0  # Normalize
y = np.array(y)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset'